In [1]:
import pandas as pd

1. Получаем данные

In [2]:
events = pd.read_csv('events.csv')
events_df = events.copy()

In [3]:
purchase = pd.read_csv('purchase.csv')
purchase_df = purchase.copy()

2. Оставляем пользователей, зарегистрированных в 2018 году; приводим дату к формату datetime; объединяем dataframes

In [4]:
mask1 = (events_df['start_time']>='2018-01-01') & (events_df['start_time']<'2019-01-01') & (events_df['event_type']=='registration')
registered2018 = events_df[mask1]['user_id'].to_list()
events_df = events_df[events_df['user_id'].isin(registered2018)]

In [5]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'], format='%Y-%m-%dT%H:%M:%S')

In [6]:
purchase_df = purchase_df[purchase_df['user_id'].isin(registered2018)]
purchase_df = purchase_df.rename(columns={"event_datetime": "start_time"})
purchase_df['start_time'] = pd.to_datetime(purchase_df['start_time'], format='%Y-%m-%dT%H:%M:%S')

In [7]:
purchase_df['event_type'] = 'purchase'
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})
total_events_df = pd.concat([events_df,purchase_df],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

3. Выделяем группы пользователей по выбранному уровню сложности

In [8]:
easy_level_users = total_events_df[total_events_df['selected_level']=="easy"]['user_id'].unique()
medium_level_users = total_events_df[total_events_df['selected_level']=="medium"]['user_id'].unique()
hard_level_users = total_events_df[total_events_df['selected_level']=="hard"]['user_id'].unique()

4. Рассчитываем для каждой группы процент оплат

In [9]:
purchase_easy = purchase_df[purchase_df['user_id'].isin(easy_level_users)]
purchase_medium = purchase_df[purchase_df['user_id'].isin(medium_level_users)]
purchase_hard = purchase_df[purchase_df['user_id'].isin(hard_level_users)]

In [10]:
percent_of_easy_purchase = purchase_easy['user_id'].nunique()/len(easy_level_users)
print("Процент пользователей, совершивших оплату, среди выбравших лёгкий уровень сложности: {:.2%}".format(percent_of_easy_purchase))

Процент пользователей, совершивших оплату, среди выбравших лёгкий уровень сложности: 7.72%


In [11]:
percent_of_medium_purchase = purchase_medium['user_id'].nunique()/len(medium_level_users)
print("Процент пользователей, совершивших оплату, среди выбравших средний уровень сложности: {:.2%}".format(percent_of_medium_purchase))

Процент пользователей, совершивших оплату, среди выбравших средний уровень сложности: 20.86%


In [12]:
percent_of_hard_purchase = purchase_hard['user_id'].nunique()/len(hard_level_users)
print("Процент пользователей, совершивших оплату, среди выбравших тяжёлый уровень сложности: {:.2%}".format(percent_of_hard_purchase))

Процент пользователей, совершивших оплату, среди выбравших тяжёлый уровень сложности: 35.39%


5. Создаём dataframes с событиями регистрации, выбора уровня сложности и оплаты

In [13]:
registration_df = total_events_df[total_events_df['event_type']=='registration']
registration_df = registration_df[["user_id", "start_time"]].rename(columns={"start_time":"registration_time"})

In [14]:
purchase_easy_df = purchase_easy.copy()
purchase_easy_df = purchase_easy_df[["user_id", "start_time"]].rename(columns={"start_time":"purchase_time"})
purchase_medium_df = purchase_medium.copy()
purchase_medium_df = purchase_medium_df[["user_id", "start_time"]].rename(columns={"start_time":"purchase_time"})
purchase_hard_df = purchase_hard.copy()
purchase_hard_df = purchase_hard_df[["user_id", "start_time"]].rename(columns={"start_time":"purchase_time"})

In [15]:
level_choice_df = total_events_df[total_events_df['event_type']=='level_choice']
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(columns={"start_time":"level_choice_time"})

5.1. Среднее время между регистрацией/выбором_уровня_сложности и оплатой у пользователей, выбравших лёгкий уровень

In [16]:
merged_df_e1 = registration_df.merge(purchase_easy_df, on="user_id", how="inner")
merged_df_e1['timedelta'] = (merged_df_e1['purchase_time']-merged_df_e1['registration_time'])
merged_df_e1['timedelta'].mean()

Timedelta('3 days 22:10:23.211640211')

In [17]:
merged_df_e2 = level_choice_df.merge(purchase_easy_df, on="user_id", how="inner")
merged_df_e2['timedelta'] = (merged_df_e2['purchase_time']-merged_df_e2['level_choice_time'])
merged_df_e2['timedelta'].mean()

Timedelta('3 days 14:58:52.941798941')

5.2. Среднее время между регистрацией/выбором_уровня_сложности и оплатой у пользователей, выбравших средний уровень

In [18]:
merged_df_m1 = registration_df.merge(purchase_medium_df, on="user_id", how="inner")
merged_df_m1['timedelta'] = (merged_df_m1['purchase_time']-merged_df_m1['registration_time'])
merged_df_m1['timedelta'].mean()

Timedelta('4 days 06:12:06.576883384')

In [19]:
merged_df_m2 = level_choice_df.merge(purchase_medium_df, on="user_id", how="inner")
merged_df_m2['timedelta'] = (merged_df_m2['purchase_time']-merged_df_m2['level_choice_time'])
merged_df_m2['timedelta'].mean()

Timedelta('3 days 23:14:13.165118679')

5.3. Среднее время между регистрацией/выбором_уровня_сложности и оплатой у пользователей, выбравших тяжелый уровень

In [20]:
merged_df_h1 = registration_df.merge(purchase_hard_df, on="user_id", how="inner")
merged_df_h1['timedelta'] = (merged_df_h1['purchase_time']-merged_df_h1['registration_time'])
merged_df_h1['timedelta'].mean()

Timedelta('3 days 14:55:19.257918552')

In [21]:
merged_df_h2 = level_choice_df.merge(purchase_hard_df, on="user_id", how="inner")
merged_df_h2['timedelta'] = (merged_df_h2['purchase_time']-merged_df_h2['level_choice_time'])
merged_df_h2['timedelta'].mean()

Timedelta('3 days 07:20:41.420814479')

Таким образом:

In [22]:
print("Среднее время между регистрацией и оплатой у пользователей, выбравших лёгкий уровень: {}".format(merged_df_e1['timedelta'].mean()))
print("Среднее время между выбором уровня сложности и оплатой у пользователей, выбравших лёгкий уровень: {}\n".format(merged_df_e2['timedelta'].mean()))
print("Среднее время между регистрацией и оплатой у пользователей, выбравших средний уровень: {}".format(merged_df_m1['timedelta'].mean()))
print("Среднее время между выбором уровня сложности и оплатой у пользователей, выбравших средний уровень: {}\n".format(merged_df_m2['timedelta'].mean()))
print("Среднее время между регистрацией и оплатой у пользователей, выбравших тяжелый уровень: {}".format(merged_df_h1['timedelta'].mean()))
print("Среднее время между выбором уровня сложности и оплатой у пользователей, выбравших тяжелый уровень: {}".format(merged_df_h2['timedelta'].mean()))

Среднее время между регистрацией и оплатой у пользователей, выбравших лёгкий уровень: 3 days 22:10:23.211640211
Среднее время между выбором уровня сложности и оплатой у пользователей, выбравших лёгкий уровень: 3 days 14:58:52.941798941

Среднее время между регистрацией и оплатой у пользователей, выбравших средний уровень: 4 days 06:12:06.576883384
Среднее время между выбором уровня сложности и оплатой у пользователей, выбравших средний уровень: 3 days 23:14:13.165118679

Среднее время между регистрацией и оплатой у пользователей, выбравших тяжелый уровень: 3 days 14:55:19.257918552
Среднее время между выбором уровня сложности и оплатой у пользователей, выбравших тяжелый уровень: 3 days 07:20:41.420814479


По итогам проведённого исследования можно сделать следующие выводы:

1. Доля пользователей, совершивших целевое действие (оплата), растёт со сложностью выбираемых вопросов.
   Среди пользователей, выбравших лёгкие вопросы, совершили оплату 7.72%.
   Пользователи, выбравшие вопросы среднего уровня, докупали пакеты вопросов уже в 20.86% случаев.
   Тогда как пользователи, выбиравшие сложные вопросы, совершали покупки в 35.39% случаев.

   Соответственно, вероятность оплаты напрямую зависит от сложности выбираемых пользователем вопросов.
   

2. Напротив, при исследовании среднего времени между регистрацией или выбором уровня сложности и совершением покупки среди групп
   пользователей, выбиравших лёгкие, средние или тяжелые вопросы, не было замечено существенной разницы.
   Да, пользователи, отвечавшие на наиболее сложные вопросы, доходили до покупки в среднем на несколько часов быстрее, но наиболее долгий путь к оплате был у пользователей, выбиравших вопросы среднего уровня, а не лёгкого, как можно было бы предположить.

   Соответственно, скорость оплаты не зависит от сложности выбираемых пользователем вопросов.